In [ ]:
<div style="text-align: center; font-family: Arial, sans-serif; padding: 20px;">
    <h2>Teachable Machine Image Model</h2>
    <button type="button" onclick="init()">Start Webcam</button>
    <br><br>
    <div id="webcam-container" style="margin-bottom: 20px;"></div>
    <div id="label-container" style="margin-bottom: 20px;"></div>
    <div id="result-container">
        <h3>Recorded Results:</h3>
        <ul id="result-list"></ul>
    </div>
</div>

<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@latest/dist/tf.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/@teachablemachine/image@latest/dist/teachablemachine-image.min.js"></script>

<script type="text/javascript">
    const URL = "./my_model/";  // Replace with your model URL
    let model, webcam, labelContainer, maxPredictions;

    // Initialize the model and webcam
    async function init() {
        const modelURL = URL + "model.json";
        const metadataURL = URL + "metadata.json";

        // Load the model and metadata
        model = await tmImage.load(modelURL, metadataURL);
        maxPredictions = model.getTotalClasses();

        // Setup webcam
        const flip = true;
        webcam = new tmImage.Webcam(200, 200, flip);
        await webcam.setup();
        await webcam.play();
        window.requestAnimationFrame(loop);

        // Append webcam to DOM
        document.getElementById("webcam-container").appendChild(webcam.canvas);

        // Create label containers for class predictions
        labelContainer = document.getElementById("label-container");
        for (let i = 0; i < maxPredictions; i++) {
            labelContainer.appendChild(document.createElement("div"));
        }

        // Start 3-second timer to record prediction
        setTimeout(() => {
            capturePrediction();
        }, 3000);
    }

    async function loop() {
        webcam.update();  // Update webcam frame
        await predict();
        window.requestAnimationFrame(loop);
    }

    // Predict and display class probabilities
    async function predict() {
        const prediction = await model.predict(webcam.canvas);
        for (let i = 0; i < maxPredictions; i++) {
            const classPrediction = prediction[i].className + ": " + prediction[i].probability.toFixed(2);
            labelContainer.childNodes[i].innerHTML = classPrediction;
        }
    }

    // Capture prediction after 3 seconds and add it to result list
    async function capturePrediction() {
        const prediction = await model.predict(webcam.canvas);
        let mostLikelyClass = '';
        let maxProbability = 0;

        for (let i = 0; i < maxPredictions; i++) {
            if (prediction[i].probability > maxProbability) {
                maxProbability = prediction[i].probability;
                mostLikelyClass = prediction[i].className;
            }
        }

        // Add to result list
        addToResultList(mostLikelyClass);
    }

    // Add captured result to list
    function addToResultList(result) {
        const resultList = document.getElementById("result-list");
        const li = document.createElement("li");
        li.textContent = result;

        // Add delete button
        const deleteBtn = document.createElement("button");
        deleteBtn.textContent = "Delete";
        deleteBtn.style.marginLeft = "10px";
        deleteBtn.onclick = () => {
            resultList.removeChild(li);  // Remove result on click
        };

        li.appendChild(deleteBtn);
        resultList.appendChild(li);
    }
</script>
